<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S71_drone_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U gtbook

In [1]:
import numpy as np
import plotly.express as px
import gtsam 

# Moving in Three Dimensions

> Motion in 3D has 6 DOF.

In [2]:
from gtbook.display import randomImages
from IPython.display import display
display(randomImages(7, 1, "steampunk", 1))

In Section 6.1, we introduced $SE(2)$, and showed how matrices can be used to represent both
rotation and translation in two dimensions.
At that time, we promised that this could be easily generalized to the case of rotation and
translation in three dimensions.  In this section, we deliver on that promise, and introduce
$SO(3)$ and $SE(3)

In addition to representing the pose of an object in 3D, for the case of drones, we are also interested in
representing velocity.
This includes the both the linear velocity of the drone, which corresponds to the velocity
of the origin of the body-attached frame, as well as the rate of change in the orientation of the
body-attached frame.

## Rotations in 3D aka SO(3)

In Section 6.1, we constructed the rotation matrix $R^0_1 \in SO(2)$
by projecting the axes of Frame 1 onto Frame 0.
The extension to 3D is indeed straightforward.
Again, we merely project the axes of Frame 1 onto Frame 0,
but in this case, each frame is equpped with aw $z$-axis:

$$
R_{1}^{0}
=\begin{bmatrix}
x_1 \cdot x_0 & y_1 \cdot x_0 & z_1 \cdot x_0 \\
x_1 \cdot y_0 & y_1 \cdot y_0 & z_1 \cdot y_0 \\
x_1 \cdot z_0 & y_1 \cdot z_0 & z_1 \cdot z_0 \\
\end{bmatrix}
$$

The expression for rotatoinal coordinate transformations is now exactly the
same as in Chapter 6.
Given a point $p$ with coordinates expressed relative to the body-attached
frame, we compute its coordinates in frame $s$:

$$
p^s = R^s_b p^b
$$

Extending the semantics of $SO(2)$ to $SO(3)$,
the columns
of $R_{b}^{s}$ represent the axes of frame $B$ in the $S$ coordinate
frame: $$R_{b}^{s}=\left[\begin{array}{ccc}
\hat{x}_{b}^{s} & \hat{y}_{b}^{s} & \hat{z}_{b}^{s}\end{array}\right]$$

The 3D rotations together with composition constitute the **special
orthogonal group** $SO(3)$. It is made up of all $3\times3$ orthonormal
matrices with determinant $+1$, with matrix multiplication implementing
composition. 
However, *3D rotations do not commute*, i.e., in general $R_{2}R_{1}\neq R_{1}R_{2}$.

It is often useful to refer to rotations about one of the coordinate axes.
These basic rotation matrices are given by

$$
R_{x,\psi}
=\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos \psi & - \sin \psi \\
0 & \sin \psi & \cos \psi 
\end{bmatrix}
~~~
R_{y,\theta}
=\begin{bmatrix}
\cos \theta &      0   & \sin \theta \\
0            &     1   &  0 \\
-\sin \theta & 0       & \cos \theta
\end{bmatrix}
~~~
R_{z,\phi}
=\begin{bmatrix}
\cos \phi &-\sin \phi & 0 \\
\sin \phi &\cos \phi & 0 \\
0 & 0   &  1
\end{bmatrix}
$$

Note, in particular, the form of $R_{z,\phi}$, the rotation about the $z$-axis.
The upper-left $2\times 2$ block of this matrix is exactly the rotation matrix $R \in SO(2)$
as introduced in Section 6.1.
This can be understood by realizing that rotation in the plane is actually
a rotation about an implicit $z$- axis that points "out of the page."



## 3D Rigid transforms aka SE(3)

In a similar way, we can immediately extend $SE(2)$ to the case
of rotation an translation in 3D.
Suppose the point $p$ is rigidly attached to frame $b$, and
we wish to determine its coordinates with respect to the reference frame $s$.
If the coordinates of $p$ with resepect to frame $b$ are given by $p^b$,
then

$$
p^{s}=R_{b}^{s}p^{b}+t_{b}^{s}
$$ 

in which $R^s_b \in SO(3)$ is the rotation matrix that specifies the orientation
of frame $b$ w.r.t. frame $s$,
and  $t^s_b \in \mathbb{R}^3$ gives the location of the origin
of frame $b$ w.r.t. frame $s$ (specified in the coordinates of frame $s$).


We denote this transform by
$T_{b}^{s}\doteq\left(R_{b}^{s},\,t_{b}^{s}\right)$. The **special
Euclidean group** $SE(3)$, with the group operation defined similarly as
in Equation
<a href="#eq:composeSE2" data-reference-type="ref" data-reference="eq:composeSE2">[eq:composeSE2]</a>.
Moreover, the group $SE(3)$ is a subgroup of a general linear group
$GL(4)$ of degree 4, by embedding the rotation and translation into a
$4\times4$ invertible matrix defined as

$$
T_{b}^{s}=\begin{bmatrix}
R_{b}^{s} & t_{b}^{s}\\
0 & 1
\end{bmatrix}

$$

Again, by embedding 3D points in a four-vector, a
3D rigid transform acting on a point can be implemented by matrix-vector
multiplication:

$$\begin{bmatrix}
R_{b}^{s} & t_{b}^{s}\\
0 & 1
\end{bmatrix}
\begin{bmatrix}
p^{b}\\
1
\end{bmatrix}
=\begin{bmatrix}
R_{b}^{s}p^{b}+t_{b}^{s}\\
1
\end{bmatrix}

$$

## Angular Velocity

The linear velocity of the origin of a moving frame is easy to understand.
This velocity is obtained merely by computing the deriviative of
the coordinate vector for the frame's origin.
The derivative of orientatino is more interesting.

We begin by introducing the family of $3 \times 3$ **skew symmetric matrices**, denoted by $so(3)$ (the relationship
between $so(3)$ and $SO(3)$ will become apparent shortly).
A matrix $S$ is said to be skew symmetric if

$S + S^T = 0$$

If we examine this equation for each element of $S$, we obtain nine
equations, 

$$ s_{i,j} = - s_{j,i}, ~~~~ i,j = 1,2,3 $$

When $i=j$, we have $s_{ii} = -s_{ii}$, which is satisfied only if $s_{ii} = 0$.
Thus, the diagonal elements of a skew symmetric matrix are all zeros.
The remaining equations constrain the off-diagonal entries, such that
every $S \in so(3)$ can be written as

$$
S = \hat{s} =
\begin{bmatrix}
0 & -s_z &s_y \\
s_z & 0 & -s_x \\
-s_y & s_x & 0
\end{bmatrix}
$$

Note that we define the operator $\hat{\cdot}$, which maps a vector $s = (s_x, s_y, s_z)$ to
a skew symmetric matrix as above.

Skew symmetric matrices play a key role in representing the angular rate of change of coordinate frames.
Let $R(t)$ be a time-varying rotation matrix.
Because $R(t) \in SO(3)$, for each time $t$ we have $I = R(t) R^T(t)$.
If we differentiate both sides of this equations, we obtain

$$
\begin{align*}
\frac{d}{dt} I &=  \frac{d}{dt} \left( R(t) R^T(t) \right)   \\
     0 &= \dot{R} R^T + R \dot{R}^T \\
\end{align*}
$$

Now, note that $(\dot{R} R^T)^T = R \dot{R}^T $. 
Thus, the matrix $\dot{R} R^T$ is a skew symmetric matrix,
and we can write

$$\hat{\omega} = \dot{R} R^T$$

which immediately gives 


$$\dot{R} = \hat{\omega} R $$

The above gives an expression for the derivative of a rotation matrix that
will be used below in our formulation of drone dynamics.
The vector $\omega$ is called the **angular velocity**,
and it is indeed the same angular velocity that you may have
learned in an introductory physics course.